## Agent Network Graph
1. Generates a network graph of nodes and edges utilizing Networkx
2. Node and Edge Data comes from the cleaned DIP Data dump
3. The network for this file is formatted in the Spring Layout
4. The nodes and edges are fully built, having all of the appropriate attributes i.e DIP Loan Amount
5. Plotly is then used to generate the graph
* 3 "Traces" are generated: The edges trace, The agents trace and The cases trace

In [4]:
import plotly.plotly as py
import os
import plotly
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
#import pyperclip
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, Event, State
#import visdcc
import colorlover as cl
from IPython.display import HTML
#import visJS2jupyter.visJS_module

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

os.getcwd()
os.chdir(os.getcwd()+'\\Data')

In [5]:
df = pd.read_csv('stem_words_group.csv')

In [7]:
len(df.index)
df.head()

,Unnamed: 0,date,stems,Total
0,1,2018-01-31,committe,9
1,2,2018-01-31,expect,9
2,3,2018-01-31,inflat,13
3,4,2018-01-31,market,7
4,5,2018-01-31,rate,7


In [2]:
df = df[~df.cases.isin(['In re Walter Investment Management Corp.'])]
len(df.index)

342

In [3]:
'''
Pulls the data in from the CSV file and organizes the data into formats that Netowrkx needs
'''
date = list(df['date'])
stems = list(df['stems'])
total = list(df['Total'])
print('Total Case and Admin Observations: ', len(date) + len(stems))

edge_list = []
for i in range(0, len(df.index)):
    edge_list.append((str(df.loc[i]['date']), str(df.loc[i]['stems']), int(df.loc[i]['Total'])))
print('Total Case Admin Pairs: ', len(edge_list))

No Loan Amount Count:  1
Row Count Original:  342
Row Count After NaN Drop:  341
Total Case and Admin Observations:  682
Total Case Admin Pairs:  341


In [4]:
'''
Actually develops the network graph using the edges and nodes given from the CSV Data
'''

G = nx.DiGraph()

G.add_nodes_from(date)
G.add_nodes_from(stems)

G.add_weighted_edges_from(edge_list, weight='weight')
#Sets the edge attributes: nx.get_edge_attributes(G, 'weight')

print('Number of Nodes: ', G.number_of_nodes())
print('Number of Edges: ', G.number_of_edges())

pos = nx.spring_layout(G)
#pos = nx.get_node_attributes(G,'pos')
#position of each node in the layout of the graph
print(isinstance(pos, dict))

nx.set_node_attributes(G, pos, 'pos')

print('First Node and Data: ', list(G.nodes.data())[0])
print('First Edge and Data: ', list(G.edges.data())[0])
#if the number of Case:Admin pairs differes from the # of edges, there are duplicates present
# Number of nodes differs from Case + Admin count -> there are duplicate entries

connections = {}
for n, nbrsdict in G.adjacency():
    #print(n, nbrsdict.keys())
    #print(n, len(nbrsdict.keys()))
    connections[n] = len(nbrsdict.keys())
print('First Node and Data: ', list(G.nodes.data())[0])
nx.set_node_attributes(G, connections, 'connections')

Number of Nodes:  453
Number of Edges:  311
True
First Node and Data:  ('In re Fairway Group Holdings Corp.', {'pos': array([0.38741037, 0.81560761])})
First Edge and Data:  ('Credit Suisse', 'In re Fairway Group Holdings Corp.', {'weight': 30611941})
First Node and Data:  ('In re Fairway Group Holdings Corp.', {'pos': array([0.38741037, 0.81560761])})


In [15]:
text_list=[]
x_pos = []
y_pos = []
for edge in G.edges.data():
    #print(edge)
    #agent = str('Agent: '+ edge[0])
    #case = str('Case: ' + edge[1])
    #print(edge[2]['weight'])
    #loan_amt = str('Loan: '+ '${:,.2f}'.format(edge[2]['weight']))
    
    x0, y0 = G.node[edge[0]]['pos']
    #print(G.node[edge[0]]['pos'])
    x1, y1 = G.node[edge[1]]['pos']
    #print(G.node[edge[1]]['pos'])
    #text_list.append([agent, case, loan_amt])
    
    x_pos += (x0, x1, None)
    y_pos += (y0, y1, None)

edge_trace = go.Scatter(
    name='Connections',
    x=x_pos,
    y=y_pos,
    line=dict(width=0.3,color='#888'),
    text=text_list,
    mode='lines',
    hoverinfo='text')
#print(edge_trace['x'])
#print(type(edge_trace['x']))

# loops through edges, pulls out the case name and admin name; finds each names node position 


In [16]:
node_x = []
node_y = []
color_list = []
text_list = []
loan_amt = []

for i in range(0, len(cases)):
    #print(cases[i])
    G.nodes.data()[cases[i]]
    node_x.append(G.node[cases[i]]['pos'][0])
    node_y.append(G.node[cases[i]]['pos'][1])
    loan = loan_amount[i]
    loan_amt.append(loan)
    #text_list.append([G.node[node]['pos'][0], G.node[node]['pos'][1], node])
    connect = G.nodes.data()[cases[i]]['connections']
    color_list.append(connect)
    text_list.append([str(cases[i]), 'Loan Amount: '+'${:,.2f}'.format(loan)])

case_trace = go.Scatter(
    name='Cases',
    x=node_x,
    y=node_y,
    text=text_list,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=loan_amt,
        size=loan_amt,
        sizeref=2.*max(loan_amt)/(50.**2),
        sizemin=8,
        sizemode='area',
        colorbar=dict(
            thickness=15,
            title='Loan Amount',
            xanchor='left',
            titleside='right',
            xpad=5
        ),
        line=dict(width=2, color='black')))

In [17]:
node_x = []
node_y = []
color_list = []
text_list = []
size_list = []

for agent in admin_agent:
    #print(G.node[node]['pos'])
    G.nodes.data()[agent]
    node_x.append(G.node[agent]['pos'][0])
    node_y.append(G.node[agent]['pos'][1])
    #text_list.append([G.node[node]['pos'][0], G.node[node]['pos'][1], node])
    connect = G.nodes.data()[agent]['connections']
    color_list.append(connect)
    size_list.append(connect)
    text_list.append([str(agent), 'Connections: '+str(connect)])

admin_trace = go.Scatter(
    name='Agents',
    x=node_x,
    y=node_y,
    text=text_list,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Blues',
        reversescale=True,
        color=color_list,
        size=size_list,
        sizeref=2.*max(size_list)/(50.**2),
        sizemin=8,
        symbol='circle-dot',
        opacity=.5,
        sizemode='area',
        colorbar=dict(
            thickness=15,
            title='Connections',
            xanchor='left',
            titleside='right',
            x=1.08
        ),
        line=dict(
            width=1,
            color = 'black'
        )))

In [18]:
'''
Combines all of the traces above into one graph that is pictured below
'''

fig = go.Figure(data=[edge_trace, case_trace, admin_trace],
             layout=go.Layout(
                title='<br>Restructuring Network',
                titlefont=dict(size=16),
                showlegend=True,
                legend=dict(x=.1, y=1.1),
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text='',
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename='network')